In [1]:
import concurrent.futures

In [2]:
NUM_STATES = 2
NUM_OBS = 2

# Indexing states
GOOD = 0
BAD = 0

# Indexing observations
POLITE = 0
RUDE = 1

In [3]:
initial = [0.6, 0.4]
a = [[0.8, 0.2], [0.6, 0.4]]
b = [[0.7, 0.3], [0.5, 0.5]]

In [4]:
obs = [POLITE, RUDE, POLITE]

In [5]:
def bm():
    beta_array = []
    
    # Step 1 - Initialisation
    final_pos = []
    for i in range(NUM_STATES):
        final_pos.append(1)
    beta_array.append(final_pos)
    
    # Step 2 - Recursion
    for k in range(len(obs)-1):
        aux_list = []
        for i in range(NUM_STATES):
            beta_i_k = 0
            for j in range(NUM_STATES):
                beta_i_k += (beta_array[0][j] * a[i][j] * b[j][obs[len(obs)-k-1]])
            aux_list.append(beta_i_k)
        beta_array = [aux_list] + beta_array
        
    return beta_array

In [6]:
beta = bm()

In [7]:
def get_prob(beta_array):
    prob_sum = 0
    for i in range(NUM_STATES):
        prob_sum += (beta_array[0][i] * b[i][obs[0]] * initial[i])
    return prob_sum

In [8]:
def fm():
    alpha_array = []
    
    # Step 1 - Initialisation
    sub_array = [];
    for i in range(NUM_STATES):
        sub_array.append(initial[i] * b[i][obs[0]])
    alpha_array.append(sub_array)
    
    # Step 2 - Recursion
    for k in range(len(obs)-1):
        aux_list = []
        n = len(alpha_array)
        for i in range(NUM_STATES):
            alpha_i_k = 0
            for j in range(NUM_STATES):
                alpha_i_k += (alpha_array[n-1][j] * a[j][i])
            alpha_i_k *= b[i][obs[k+1]]
            aux_list.append(alpha_i_k);
        alpha_array = alpha_array + [aux_list]
    
    return alpha_array        

In [9]:
alpha = fm()

In [10]:
def gamma(alpha, beta):
    gamma_array = []
    for j in range(len(obs)):
        sigma_alphaT = 0
        for i in range(NUM_STATES):
            sigma_alphaT += alpha[len(obs)-1][i]
        aux_list = []
        for i in range(NUM_STATES):
            aux_list.append((alpha[j][i] * beta[j][i])/sigma_alphaT)
        gamma_array += [aux_list]
    return gamma_array

In [11]:
gm = gamma(alpha, beta)
gm

[[0.6559151975511591, 0.34408480244884077],
 [0.6397596508134459, 0.3602403491865541],
 [0.7868601553199931, 0.21313984468000685]]

In [12]:
def reestimate_output(gamma_array):
    new_b = []
    for i in range(NUM_STATES):
        sum_i = 0
        for j in range(len(obs)):
            sum_i += gamma_array[j][i]
        
        aux_list = []
        for j in range(NUM_OBS):
            sum_j = 0
            for k in range(len(obs)):
                if (obs[k] == j):
                    sum_j += gamma_array[k][i]
            aux_list.append(sum_j/sum_i)
        new_b.append([aux_list])
    return new_b    

In [14]:
# def transition_reestimate(a, b, alpha, beta, gamma):
#     new_a = []
#     for x in range(NUM_STATES):
#         aux_list = []
#         for y in range(NUM_STATES):
#             sum_nr = 0
#             sum_dr = 0
#             for t in range(len(obs)-1):
#                 sum_nr += (alpha[t][x] * beta[t+1][y] * b[y][obs[t+1]])
#                 sum_dr += gamma[t][x]
#             sum_nr *= a[x][y]
#             sum_dr += gamma[len(obs)-1][x]
#             aux_list.append(sum_nr / sum_dr)
#         new_a.append([aux_list])
#     return new_a

In [15]:
def get_si(a, b, alpha, beta, prob):
    si = []
    for i in range(NUM_STATES):
        si_i = []
        for j in range(NUM_STATES):
            si_i_j = []
            for t in range(len(obs)-1):
                si_i_j_t = alpha[t][i] * a[i][j] * b[j][obs[t+1]] * beta[t+1][j] / prob
                si_i_j.append(si_i_j_t)
            si_i.append(si_i_j)
        si.append(si_i)
    return si

In [16]:
si = get_si(a, b, alpha, beta, get_prob(beta))
si

[[[0.47140184796780227, 0.5428263703871663],
  [0.184513349583357, 0.09693328042627972]],
 [[0.16835780284564367, 0.244033784932827],
  [0.17572699960319718, 0.11620656425372716]]]

In [17]:
def reestimate_transition(gamma, si):
    new_a = []
    for x in range(NUM_STATES):
        aux_list = []
        for y in range(NUM_STATES):
            sum_nr = 0
            sum_dr = 0
            for t in range(len(obs)-1):
                sum_nr += si[x][y][t]
                sum_dr += gamma[t][x]
            aux_list.append(sum_nr / sum_dr)
        new_a.append([aux_list])
    return new_a

In [19]:
def parallel():
    # Parallel processing using Threadpools
    with concurrent.futures.ThreadPoolExecutor() as executor:
        alpha_array = executor.submit(fm).result()
        beta_array = executor.submit(bm).result()
    
    gamma_array = gamma(alpha_array, beta_array)
    si = get_si(a, b, alpha_array, beta_array, get_prob(beta_array))
    print("New output probabilities:")
    print(reestimate_output(gamma_array))
    print("New Transition probabilities:")
    print(reestimate_transition(gamma_array, si))    

In [20]:
parallel()

New output probabilities:
[[[0.692797648211661, 0.3072023517883391]], [[0.607352486870559, 0.39264751312944085]]]
New Transition probabilities:
[[[0.7827798923743274, 0.21722010762567273]], [[0.585513078470825, 0.41448692152917527]]]
